# Instâncias reais

* **timestamp**: observations timestamps loaded into pandas DataFrame as its index;
* **P-PDG**: pressure variable at the Permanent Downhole Gauge (PDG);
* **P-TPT**: pressure variable at the Temperature and Pressure Transducer (TPT);
* **T-TPT**: temperature variable at the Temperature and Pressure Transducer (TPT);
* **P-MON-CKP**: pressure variable upstream of the production choke (CKP);
* **T-JUS-CKP**: temperature variable downstream of the production choke (CKP);
* **P-JUS-CKGL**: pressure variable upstream of the gas lift choke (CKGL);
* **T-JUS-CKGL**: temperature variable upstream of the gas lift choke (CKGL);
* **QGL**: gas lift flow rate;
* **class**: observations labels associated with three types of periods (normal, fault transient, and faulty steady state).


* **label**: instance label (event type);
* **well**: well name. Hand-drawn and simulated instances have fixed names. Real instances have names masked with incremental id;
* **id**: instance identifier. Hand-drawn and simulated instances have incremental id. Each real instance has an id generated from its first timestamp.

https://github.com/petrobras/3W

In [1]:
import sys
import os
import pandas as pd
import numpy as np

import joblib
import pickle

from pyod.models.ecod import ECOD

from pyod.models.suod import SUOD

from pyod.models.xgbod import XGBOD

from pyod.models.loda import LODA

from pyod.models.pca import PCA




import time
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import SGDOneClassSVM
from sklearn.pipeline import make_pipeline
from sklearn.svm import OneClassSVM

from sklearn.model_selection import GroupKFold

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.metrics  import average_precision_score, make_scorer, roc_curve,f1_score, precision_score, recall_score, fbeta_score, auc, roc_auc_score, accuracy_score, confusion_matrix, classification_report,precision_recall_curve
import seaborn as sns

import matplotlib.pyplot as plt

from sklearn.model_selection import TimeSeriesSplit

### Function to calculate metrics

In [2]:
def metrics(y_test, y_pred):
    cm = list()
    cm = confusion_matrix(y_test, y_pred)
    cm_df = pd.DataFrame(cm)                      
    plt.figure(figsize=(8,6))  
    sns.heatmap(cm_df, annot=True)
    print("Classification Report: \n", classification_report(y_test, y_pred, digits=5))  


    TN = cm[0][0]
    FP = cm[0][1]
    FN = cm[1][0]
    TP = cm[1][1]

    print("Specificity:", TN/(TN+FP))


    roc_auc = roc_auc_score(y_test, y_pred, multi_class = 'ovr', average=None)
    gini = 2*roc_auc -1
    print("Gini: ",gini)
    print("ROC AUC:: ",roc_auc)

## Read data

In [3]:
df = pd.read_parquet('./data/real_instances.parquet.gzip') 

In [4]:
df.shape

(14516197, 12)

In [5]:
df.head()

,label,well,id,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,P-JUS-CKGL,T-JUS-CKGL,QGL,class
timestamp,,,,,,,,,,,,
2017-02-01 02:02:07,0,WELL-00001,20170201020207,0.0,10092110.0,119.0944,1609800.0,84.59782,1564147.0,NaN,0.0,0.0
2017-02-01 02:02:08,0,WELL-00001,20170201020207,0.0,10092000.0,119.0944,1618206.0,84.58997,1564148.0,NaN,0.0,0.0
2017-02-01 02:02:09,0,WELL-00001,20170201020207,0.0,10091890.0,119.0944,1626612.0,84.58213,1564148.0,NaN,0.0,0.0
2017-02-01 02:02:10,0,WELL-00001,20170201020207,0.0,10091780.0,119.0944,1635018.0,84.57429,1564148.0,NaN,0.0,0.0
2017-02-01 02:02:11,0,WELL-00001,20170201020207,0.0,10091670.0,119.0944,1643424.0,84.56644,1564148.0,NaN,0.0,0.0


In [6]:
df.id.nunique()

1013

In [7]:
df['label'].value_counts()

label
0    9903155
4    2462076
3     569152
5     552529
7     466338
1     312136
2     194233
6      56578
Name: count, dtype: int64

In [8]:
df['class'].value_counts()

class
0.0      10003293
4.0       2462076
3.0        569152
105.0      317565
107.0      283262
101.0       95658
102.0       65130
7.0         25870
2.0         16100
5.0         13031
6.0         12951
1.0         10417
106.0        6252
Name: count, dtype: int64

In [9]:
pd.crosstab(df['well'], df['label'])

label,0,1,2,3,4,5,6,7
well,,,,,,,,
WELL-00001,1652442,58468,0,17976,261457,0,0,73033
WELL-00002,3641159,24017,9121,0,807601,0,47869,0
WELL-00003,463543,0,35406,0,0,0,0,0
WELL-00004,85505,0,0,0,307674,0,8709,0
WELL-00005,979611,0,0,0,271525,0,0,0
WELL-00006,2058403,229651,0,0,0,0,0,305517
WELL-00007,14370,0,0,0,71651,0,0,0
WELL-00008,1008122,0,0,0,0,0,0,0
WELL-00009,0,0,6738,0,0,0,0,0


In [10]:
pd.crosstab(df['class'], df['label'])

label,0,1,2,3,4,5,6,7
class,,,,,,,,
0.0,9439612,195376,52017,0,0,127930,36319,152039
1.0,0,10417,0,0,0,0,0,0
2.0,0,0,16100,0,0,0,0,0
3.0,0,0,0,569152,0,0,0,0
4.0,0,0,0,0,2462076,0,0,0
5.0,0,0,0,0,0,13031,0,0
6.0,0,0,0,0,0,0,12951,0
7.0,0,0,0,0,0,0,0,25870
101.0,0,95658,0,0,0,0,0,0


In [11]:
df.drop_duplicates(subset=['id'], keep='last').groupby("label")["id"].count()

label
0    588
1      5
2     22
3     32
4    344
5     11
6      6
7      5
Name: id, dtype: int64

In [12]:
df.isnull().sum()/len(df)

label         0.000000
well          0.000000
id            0.000000
P-PDG         0.000580
P-TPT         0.008007
T-TPT         0.008007
P-MON-CKP     0.077959
T-JUS-CKP     0.113279
P-JUS-CKGL    0.071501
T-JUS-CKGL    1.000000
QGL           0.191125
class         0.043775
dtype: float64

## Dataprep

In [13]:
df['target'] = 1
df.loc[(df['label']==0)&(df['class']==0), "target"] = 0

In [14]:
df.target.value_counts()

target
0    9439612
1    5076585
Name: count, dtype: int64

In [15]:
df.target.value_counts(normalize=True)

target
0    0.650281
1    0.349719
Name: proportion, dtype: float64

In [16]:
df2 = df.drop(['T-JUS-CKGL'], axis=1)
df2 = df2.dropna()

In [17]:
del df

In [18]:
df2.shape

(11108977, 12)

In [19]:
df2.target.value_counts(normalize=True)

target
0    0.663057
1    0.336943
Name: proportion, dtype: float64

In [20]:
df3 = df2.reset_index()
df3.head()

,timestamp,label,well,id,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,P-JUS-CKGL,QGL,class,target
0,2017-02-01 02:02:07,0,WELL-00001,20170201020207,0.0,10092110.0,119.0944,1609800.0,84.59782,1564147.0,0.0,0.0,0
1,2017-02-01 02:02:08,0,WELL-00001,20170201020207,0.0,10092000.0,119.0944,1618206.0,84.58997,1564148.0,0.0,0.0,0
2,2017-02-01 02:02:09,0,WELL-00001,20170201020207,0.0,10091890.0,119.0944,1626612.0,84.58213,1564148.0,0.0,0.0,0
3,2017-02-01 02:02:10,0,WELL-00001,20170201020207,0.0,10091780.0,119.0944,1635018.0,84.57429,1564148.0,0.0,0.0,0
4,2017-02-01 02:02:11,0,WELL-00001,20170201020207,0.0,10091670.0,119.0944,1643424.0,84.56644,1564148.0,0.0,0.0,0


In [21]:
df3['well'].value_counts()

well
WELL-00002    4518972
WELL-00006    2588359
WELL-00001    2062763
WELL-00014     699734
WELL-00010     601704
WELL-00017     382288
WELL-00018      87162
WELL-00007      86021
WELL-00015      54811
WELL-00016      20465
WELL-00009       6698
Name: count, dtype: int64

In [22]:
df3['id'].value_counts()

id
20180617181315    188331
20140314135248    141046
20180620155728    112748
20170731180930     95415
20140319031616     89674
                   ...  
20170917120016      6750
20170313160804      6698
20180404222255      3891
20180426105723      3172
20180426102531      1606
Name: count, Length: 740, dtype: int64

In [23]:
df2.head()

,label,well,id,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,P-JUS-CKGL,QGL,class,target
timestamp,,,,,,,,,,,,
2017-02-01 02:02:07,0,WELL-00001,20170201020207,0.0,10092110.0,119.0944,1609800.0,84.59782,1564147.0,0.0,0.0,0
2017-02-01 02:02:08,0,WELL-00001,20170201020207,0.0,10092000.0,119.0944,1618206.0,84.58997,1564148.0,0.0,0.0,0
2017-02-01 02:02:09,0,WELL-00001,20170201020207,0.0,10091890.0,119.0944,1626612.0,84.58213,1564148.0,0.0,0.0,0
2017-02-01 02:02:10,0,WELL-00001,20170201020207,0.0,10091780.0,119.0944,1635018.0,84.57429,1564148.0,0.0,0.0,0
2017-02-01 02:02:11,0,WELL-00001,20170201020207,0.0,10091670.0,119.0944,1643424.0,84.56644,1564148.0,0.0,0.0,0


In [24]:
VARS = ['P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP', 'T-JUS-CKP', 'P-JUS-CKGL', 'QGL']
VARS_AUX = ['timestamp', 'well', 'target']
data = df3[VARS+VARS_AUX]

In [25]:
data.head()

,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,P-JUS-CKGL,QGL,timestamp,well,target
0,0.0,10092110.0,119.0944,1609800.0,84.59782,1564147.0,0.0,2017-02-01 02:02:07,WELL-00001,0
1,0.0,10092000.0,119.0944,1618206.0,84.58997,1564148.0,0.0,2017-02-01 02:02:08,WELL-00001,0
2,0.0,10091890.0,119.0944,1626612.0,84.58213,1564148.0,0.0,2017-02-01 02:02:09,WELL-00001,0
3,0.0,10091780.0,119.0944,1635018.0,84.57429,1564148.0,0.0,2017-02-01 02:02:10,WELL-00001,0
4,0.0,10091670.0,119.0944,1643424.0,84.56644,1564148.0,0.0,2017-02-01 02:02:11,WELL-00001,0


In [26]:
data.dtypes

P-PDG                float64
P-TPT                float64
T-TPT                float64
P-MON-CKP            float64
T-JUS-CKP            float64
P-JUS-CKGL           float64
QGL                  float64
timestamp     datetime64[ns]
well                  object
target                 int64
dtype: object

In [27]:
data['well'].unique()

array(['WELL-00001', 'WELL-00002', 'WELL-00006', 'WELL-00007',
       'WELL-00009', 'WELL-00010', 'WELL-00014', 'WELL-00015',
       'WELL-00016', 'WELL-00017', 'WELL-00018'], dtype=object)

In [28]:
# Lista para armazenar os dados de treinamento e teste
train = list()
test = list()

data = data.sort_values(by=['well', 'timestamp'])

# Iterar sobre cada poco
for poco_id, poco_data in data.groupby('well'):
    # Calcular o índice para dividir os dados em treinamento e teste
    split_index = int(0.80 * len(poco_data))
    
    # Dividir os dados do poco em treinamento e teste
    poco_train = poco_data.iloc[:split_index]
    poco_test = poco_data.iloc[split_index:]
    
    # Adicionar os dados de treinamento e teste à lista
    train.append(poco_train)
    test.append(poco_test)

# Concatenar os dados de treinamento e teste
train = pd.concat(train)
test = pd.concat(test)

# Verificar o tamanho dos conjuntos de treinamento e teste
print("Tamanho do conjunto de treinamento:", len(train))
print("Tamanho do conjunto de teste:", len(test))


Tamanho do conjunto de treinamento: 8887177
Tamanho do conjunto de teste: 2221800


In [29]:
print(f'Treino Poços: {train.well.unique()}')
print(f'Teste Poços: {test.well.unique()}')
print(f'Tamanho do treino: {len(train)}')
print(f'Tamanho do teste: {len(test)}')

Treino Poços: ['WELL-00001' 'WELL-00002' 'WELL-00006' 'WELL-00007' 'WELL-00009'
 'WELL-00010' 'WELL-00014' 'WELL-00015' 'WELL-00016' 'WELL-00017'
 'WELL-00018']
Teste Poços: ['WELL-00001' 'WELL-00002' 'WELL-00006' 'WELL-00007' 'WELL-00009'
 'WELL-00010' 'WELL-00014' 'WELL-00015' 'WELL-00016' 'WELL-00017'
 'WELL-00018']
Tamanho do treino: 8887177
Tamanho do teste: 2221800


In [30]:
# Verificar a proporção de anomalias
print(f"Proporção de anomalias no conjunto de treino: {train.target.mean():.2%}")
print(f"Proporção de anomalias no conjunto de teste: {test.target.mean():.2%}")

Proporção de anomalias no conjunto de treino: 33.92%
Proporção de anomalias no conjunto de teste: 32.77%


In [31]:
train.head()

,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,P-JUS-CKGL,QGL,timestamp,well,target
7365884,0.0,18433410.0,116.8718,9397031.0,74.80031,-300917.8,0.0,2014-01-24 09:33:03,WELL-00001,1
7365885,0.0,18433410.0,116.8718,9397031.0,74.80031,-300917.8,0.0,2014-01-24 09:33:04,WELL-00001,1
7365886,0.0,18433410.0,116.8718,9397032.0,74.80031,-300917.8,0.0,2014-01-24 09:33:05,WELL-00001,1
7365887,0.0,18433410.0,116.8718,9397033.0,74.80031,-300917.8,0.0,2014-01-24 09:33:06,WELL-00001,1
7365888,0.0,18433410.0,116.8718,9397033.0,74.80031,-300917.8,0.0,2014-01-24 09:33:07,WELL-00001,1


In [32]:
train.tail()

,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,P-JUS-CKGL,QGL,timestamp,well,target
11091539,0.0,8431843.0,109.8008,1452053.0,74.09579,9467507.0,0.0,2019-04-03 10:09:23,WELL-00018,1
11091540,0.0,8431876.0,109.8004,1452842.0,74.09563,9467529.0,0.0,2019-04-03 10:09:24,WELL-00018,1
11091541,0.0,8431910.0,109.8000,1453631.0,74.09549,9467550.0,0.0,2019-04-03 10:09:25,WELL-00018,1
11091542,0.0,8431892.0,109.7996,1454419.0,74.09534,9467570.0,0.0,2019-04-03 10:09:26,WELL-00018,1
11091543,0.0,8431875.0,109.7992,1455208.0,74.09519,9467592.0,0.0,2019-04-03 10:09:27,WELL-00018,1


In [33]:
test.head()

,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,P-JUS-CKGL,QGL,timestamp,well,target
1239889,-6887865.0,9682304.0,118.4759,1873020.0,84.63268,-140331.8,0.0,2017-08-23 19:43:34,WELL-00001,0
1239890,-6887865.0,9682139.0,118.4759,1881098.0,84.63499,-140331.8,0.0,2017-08-23 19:43:35,WELL-00001,0
1239891,-6887865.0,9681974.0,118.4759,1889175.0,84.63731,-140331.8,0.0,2017-08-23 19:43:36,WELL-00001,0
1239892,-6887865.0,9681809.0,118.4759,1897253.0,84.63963,-140331.8,0.0,2017-08-23 19:43:37,WELL-00001,0
1239893,-6887865.0,9681644.0,118.4759,1905330.0,84.64195,-140331.8,0.0,2017-08-23 19:43:38,WELL-00001,0


In [34]:
test.tail()

,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,P-JUS-CKGL,QGL,timestamp,well,target
11108972,0.0,8489354.0,109.7213,1496222.0,73.38219,9706965.0,0.0,2019-04-03 14:59:56,WELL-00018,1
11108973,0.0,8489349.0,109.7212,1495828.0,73.38310,9706899.0,0.0,2019-04-03 14:59:57,WELL-00018,1
11108974,0.0,8489344.0,109.7210,1495433.0,73.38401,9706834.0,0.0,2019-04-03 14:59:58,WELL-00018,1
11108975,0.0,8489338.0,109.7209,1495039.0,73.38493,9706768.0,0.0,2019-04-03 14:59:59,WELL-00018,1
11108976,0.0,8489333.0,109.7207,1494645.0,73.38583,9706702.0,0.0,2019-04-03 15:00:00,WELL-00018,1


In [35]:
train.to_parquet('./data/train_temp.parquet.gzip',
              compression='gzip')

test.to_parquet('./data/test_temp.parquet.gzip',
              compression='gzip')

In [36]:
del data
del df2

In [37]:
VARS = ['P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP', 'T-JUS-CKP', 'P-JUS-CKGL', 'QGL']

In [38]:
train[VARS].describe()

,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,P-JUS-CKGL,QGL
count,8.887177e+06,8.887177e+06,8.887177e+06,8.887177e+06,8.887177e+06,8.887177e+06,8.887177e+06
mean,1.235287e+07,2.813876e+07,1.081664e+02,4.674648e+06,7.623365e+01,1.140796e+08,1.625444e-01
std,1.787440e+07,2.052284e+08,2.159599e+01,3.431340e+06,3.564559e+01,4.279944e+08,4.450774e-01
min,-1.259876e+08,4.728049e+06,7.236000e+00,7.528369e+05,-2.020000e+00,-4.976717e+05,0.000000e+00
25%,0.000000e+00,8.556373e+06,1.164812e+02,1.803939e+06,6.165615e+01,2.321333e+06,0.000000e+00
50%,0.000000e+00,1.420617e+07,1.169635e+02,2.542643e+06,7.284150e+01,2.338973e+06,0.000000e+00
75%,2.414653e+07,1.682950e+07,1.177312e+02,6.175858e+06,7.800410e+01,4.558287e+06,0.000000e+00
max,4.485805e+07,2.941990e+09,1.196061e+02,1.235933e+07,1.730961e+02,2.027865e+09,4.146514e+00


Padronização dos dados

In [39]:
# scale = StandardScaler() 
scale = MinMaxScaler()

X_train = scale.fit_transform(train[VARS])

X_test = scale.transform(test[VARS])

X_train = pd.DataFrame(X_train, columns=VARS)
X_test = pd.DataFrame(X_test, columns=VARS)

In [40]:
with open('./models/scale_temp.pkl', 'wb') as file: 
    pickle.dump(scale, file)

In [3]:
file_scale = "./models/scale_temp.pkl"

scale = joblib.load(file_scale)

In [4]:
train = pd.read_parquet('./data/train_temp.parquet.gzip') 
test = pd.read_parquet('./data/test_temp.parquet.gzip') 

In [5]:
train.head()

,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,P-JUS-CKGL,QGL,timestamp,well,target
7365884,0.0,18433410.0,116.8718,9397031.0,74.80031,-300917.8,0.0,2014-01-24 09:33:03,WELL-00001,1
7365885,0.0,18433410.0,116.8718,9397031.0,74.80031,-300917.8,0.0,2014-01-24 09:33:04,WELL-00001,1
7365886,0.0,18433410.0,116.8718,9397032.0,74.80031,-300917.8,0.0,2014-01-24 09:33:05,WELL-00001,1
7365887,0.0,18433410.0,116.8718,9397033.0,74.80031,-300917.8,0.0,2014-01-24 09:33:06,WELL-00001,1
7365888,0.0,18433410.0,116.8718,9397033.0,74.80031,-300917.8,0.0,2014-01-24 09:33:07,WELL-00001,1


In [6]:
VARS = ['P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP', 'T-JUS-CKP', 'P-JUS-CKGL', 'QGL']

In [7]:
X_train_p = scale.transform(train[VARS])

X_test_p = scale.transform(test[VARS])

X_train_p = pd.DataFrame(X_train_p, columns=VARS)
X_test_p = pd.DataFrame(X_test_p, columns=VARS)

In [8]:
X_train_p.head()

,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,P-JUS-CKGL,QGL
0,0.737435,0.004666,0.975667,0.744772,0.438682,0.000097,0.0
1,0.737435,0.004666,0.975667,0.744772,0.438682,0.000097,0.0
2,0.737435,0.004666,0.975667,0.744772,0.438682,0.000097,0.0
3,0.737435,0.004666,0.975667,0.744772,0.438682,0.000097,0.0
4,0.737435,0.004666,0.975667,0.744772,0.438682,0.000097,0.0


In [115]:
# for i in VARS:
#     train_p[i] = train_p[i].astype(float)

# for i in VARS:
#     test_p[i] = test_p[i].astype(float)